<a href="https://colab.research.google.com/github/karimghabbour/VirtualBuddy_RAG/blob/main/HPE_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Download GPT4ALL Model

In [1]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

--2024-03-28 14:12:58--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 104.26.1.159, 104.26.0.159, 172.67.71.169, ...
Connecting to gpt4all.io (gpt4all.io)|104.26.1.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  5.71MB/s    in 6m 48s  

2024-03-28 14:19:47 (8.84 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin’ saved [3785248281/3785248281]



#Step 2:Download Documents to be Accessed

In [2]:
#Mount Google Drive For Access
from google.colab import drive
drive.mount('/content/drive')

# Define the base path to the folder containing your PDFs
base_path = '/content/drive/My Drive/BasicRAG/'

# List of your PDF files
pdf_files = [
    'Resources inside HPE Barcelona Hub.pdf',
    'uncover-hidden-insights-create-business-advantage.pdf',
    'HPE GreenLake FAQ for technology innovators frequently asked questions-a00107144enw.pdf',
    'HPE Tech Care Service-a00108652enw.pdf'
]

# Full paths to the PDF files
pdf_paths = [base_path + pdf for pdf in pdf_files]



Mounted at /content/drive


# Download Extra Packages

In [ ]:
!pip install pymupdf pygpt4all sentence_transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.0/362.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.0/246.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━ 320.8/410.6 MB 1.8 MB/s eta 0:00:52

# Download Llama Index Package

In [ ]:
!pip install langchain
!pip install llama-index
%pip install llama-index-embeddings-langchain
%pip install llama-index-llms-langchain



# Document Set up

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from llama_index.core.node_parser import SimpleFileNodeParser
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.langchain import LangChainLLM #instead of LLMPredictor
from llama_index.core import ServiceContext
from llama_index.core import StorageContext
from llama_index.core import download_loader
from llama_index.core import PromptHelper



In [ ]:
# Assuming PyMuPDFReader and the list of pdf_paths are already defined

documents = []
for file_path in pdf_paths:  # Iterate over each PDF file path
    # Load the documents with metadata
    loaded_docs = PyMuPDFReader().load(file_path=file_path, metadata=True)
    # Extend the main documents list with the loaded documents
    documents.extend(loaded_docs)

# Ensure document texts are correctly formatted if needed
for doc in documents:
    if isinstance(doc.text, bytes):
        doc.text = doc.text.decode()
